In [1]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import os
#from utils.preprocess import extract_findings
from utils.preprocess import *
from utils.negationprocessing import *
from utils.createreports import *
from utils.fakereporting import *


In [2]:
rootdir="/Users/razimahmood/envpytorch/pytorchprojects"
ml_gtdir = "/Users/razimahmood/envpytorch/pytorchprojects/summer23_research/ml_gt_indiana/"
indir=rootdir+"/allreportsIndiana/"
# this is the input directory for all the text & image data from Kaggle
outdir=rootdir+"/allreportsIndianaout"

outdir1=outdir+"/removerandom/"
outdir2=outdir+"/addrandom/"
outdir3=outdir+"/exchange/"
outdir4=outdir+"/reversefinding/"


#radiologist_gtdir = "/Users/razimahmood/envpytorch/pytorchprojects/summer23research/radiologist_gt/"

gtfile="/Users/razimahmood/envpytorch/pytorchprojects/indiana_reports.csv"
#dicomdir="/Users/razimahmood/envpytorch/pytorchprojects/DICOMtest"

dicomdir = "/Users/razimahmood/envpytorch/pytorchprojects/dicom_images/images_normalized"



### Run preprocessing to extract all necessary information
## 1. extract_findings: extracts findings using the "Problems" column in indiana_reports.csv from Kaggle
### findings -> are from kaggle Problemmmap
### sentence_findingMap -> from chexpert labels
## 2. process_mapping : creates the following maps needed for the report generation:
### a. problem_map -> records all sentences across reports that contain the finding indexed by finding as the key (this uses the finding labels (cheXpert provided) at the sentence level)
### b. filename_map -> records all findings within a report indexed by the reportname as the key
### c. origsentenceMap ->records all sentences within a report as an array, reflecting the order in which they came 
### d. report_to_finding_to_sentenceMap ->double hashmap, reportname->finding->sentence containing the finding. The first hahsmap index(key) is the reportname, the second level index key is the finding. 
## 3. capitalize_entries ; capitalizes all sentences to enable Spacy analysis (it expects a sentence). Produces:
### revised_sentenceMap ->is the same as origsentenceMap except that the first letter of the sentences is capitalized
## 4. extract_neg_map: detects which findings are negated. Produceds:
### neg_map-> a hashMap that records findings->list of sentences in which this finding is in the negated sense. This is done at the sentence level regardless of which report it comes from.
## 5. process_dicomfilenames: record the patient id, the reports associated with the patient and the imaging files associated with the reports. Created a double hashmap:
### files_per_patient: patientid->reportname->list of images covered by the report. 
### e.g. 0106 {'CXR1156.txt': ['1156_IM-0106-1001.dcm.png', '1156_IM-0106-2001.dcm.png'], 'CXR1157.txt': ['1157_IM-0106-1001.dcm.png', '1157_IM-0106-2001.dcm.png']}, here 0106 is the patientID, CSR1156.txt is the report name, and 1156_IM-0106-1001.dcm.png is one of the views associated with the report.

In [32]:
#this is preprocessing to generate reports
#findings=extract_findings(gtfile)
findings,sentence_findingMap=extract_findings(gtfile,ml_gtdir+"Chexpertlabels.txt")
problem_map,filename_map,report_to_finding_to_sentenceMap,origsentenceMap=process_mapping(indir, findings,sentence_findingMap)
revised_sentenceMap=capitalize_entries(origsentenceMap)
neg_map=extract_neg_map(indir,nlp0,neg_model,nlpmodel)
files_per_patient=process_dicomfilenames(dicomdir)

In [33]:
#this is preprocessing to generate reports
print_k_lines(sentence_findingMap,5)

0 the lungs are clear, and without focal air space opacity. {'lung', 'opacity'}
1 emphysematous changes are present. {'emphysema'}
2 within the right lung, there are increased interstitial opacities within the medial right lung base and right upper lobe, with patchy airspace opacity within the right lung apex. {'lung', 'opacity'}
3 no focal consolidation, suspicious pulmonary opacity, pleural effusion, or pneumothorax. {'pleural effusion', 'consolidation', 'pneumothorax', 'opacity', 'thorax', 'pleura'}
4 low lung volumes. {'lung'}


In [34]:
print_k_lines(files_per_patient,5)

0 0272 {'CXR1425.txt': ['1425_IM-0272-1001.dcm.png', '1425_IM-0272-1002.dcm.png'], 'CXR1426.txt': ['1426_IM-0272-1001.dcm.png', '1426_IM-0272-2001.dcm.png']}
1 2200 {'CXR616.txt': ['616_IM-2200-1001.dcm.png', '616_IM-2200-3001.dcm.png'], 'CXR617.txt': ['617_IM-2200-1001.dcm.png', '617_IM-2200-1002.dcm.png'], 'CXR613.txt': ['613_IM-2200-1001.dcm.png', '613_IM-2200-2001.dcm.png'], 'CXR614.txt': ['614_IM-2200-1001.dcm.png', '614_IM-2200-4001.dcm.png'], 'CXR615.txt': ['615_IM-2200-1001.dcm.png', '615_IM-2200-1002.dcm.png']}
2 1412 {'CXR3039.txt': ['3039_IM-1412-1001.dcm.png', '3039_IM-1412-2001.dcm.png']}
3 1505 {'CXR3189.txt': ['3189_IM-1505-1001.dcm.png'], 'CXR319.txt': ['319_IM-1505-1001.dcm.png', '319_IM-1505-1002.dcm.png'], 'CXR3194.txt': ['3194_IM-1505-1001.dcm.png', '3194_IM-1505-2001.dcm.png'], 'CXR3191.txt': ['3191_IM-1505-1001.dcm.png', '3191_IM-1505-2001.dcm.png'], 'CXR3192.txt': ['3192_IM-1505-1001.dcm.png', '3192_IM-1505-2001.dcm.png'], 'CXR3190.txt': ['3190_IM-1505-1001.dcm.p

In [7]:
print_k_lines(problem_map,2) #finding->sentences (from Chexpert-provided labels)

0 medical device set()
1 hernia, diaphragmatic set()


In [8]:
print_k_lines(filename_map,4) #report ->findings (from Kaggle)

0 CXR1511.txt {'normal', 'heart', 'lung', 'mediastinum'}
1 CXR3460.txt {'lung', 'infiltrate', 'pneumothorax', 'thorax', 'heart', 'mediastinum'}
2 CXR1277.txt {'pleural effusion', 'normal', 'consolidation', 'lung', 'pneumothorax', 'thorax', 'pleura'}
3 CXR1263.txt {'normal', 'lung'}


In [9]:
print_k_lines(origsentenceMap,4) #sentences in each report

0 CXR1511.txt ['both lungs are clear and expanded.', 'heart and mediastinum normal.']
1 CXR3460.txt ['the heart and mediastinum are unremarkable.', 'the lungs are clear without infiltrate.', 'there is no effusion or pneumothorax.']
2 CXR1277.txt ['pa and lateral views the chest were obtained.', 'the cardiomediastinal silhouette is normal in size and configuration.', 'the lungs are well aerated.', 'no pneumothorax, pleural effusion, or lobar air space consolidation.', 'xxxx right middle lobe collapse appears less distinct than on prior study.']
3 CXR1263.txt ['cardiac and mediastinal contours are within normal limits.', 'the lungs are clear.', 'bony structures are intact.']


In [10]:
print_k_lines(report_to_finding_to_sentenceMap,2) #report->finding->sentences

0 CXR1511.txt {'lung': {'both lungs are clear and expanded.'}, 'heart': {'heart and mediastinum normal.'}, 'normal': {'heart and mediastinum normal.'}, 'mediastinum': {'heart and mediastinum normal.'}}
1 CXR3460.txt {'heart': {'the heart and mediastinum are unremarkable.'}, 'mediastinum': {'the heart and mediastinum are unremarkable.'}, 'lung': {'the lungs are clear without infiltrate.'}, 'infiltrate': {'the lungs are clear without infiltrate.'}, 'pneumothorax': {'there is no effusion or pneumothorax.'}, 'thorax': {'there is no effusion or pneumothorax.'}}


In [11]:
print_k_lines(revised_sentenceMap,2) #this is for pretty printing and also language analysis for negations

0 CXR1511.txt ['Both lungs are clear and expanded.', 'Heart and mediastinum normal.']
1 CXR3460.txt ['The heart and mediastinum are unremarkable.', 'The lungs are clear without infiltrate.', 'There is no effusion or pneumothorax.']


In [12]:
print_k_lines(neg_map,1) #the result of negation detection

0 effusion {'no focal consolidation, effusion, or pneumothorax identified.', 'the lungs are clear without evidence of acute infiltrate or effusion.', 'no focal consolidations, effusions, or pneumothoraces.', 'no significant effusion.', 'no pneumothorax or effusion.', 'no effusions no pneumothorax.', 'there is no acute infiltrate or effusion.', 'negative for effusion, pneumothorax, or focal airspace consolidation.', 'no focal infiltrate or effusion.', 'no large effusion or visualized pneumothorax.', 'no large effusion or airspace disease.', 'no evidence of focal infiltrate or effusion.', 'there is no pneumothorax or effusion.', 'no focal consolidation, effusion, or pneumothorax.', 'no focal airspace consolidation, large effusion, or appreciable pneumothorax.', 'no large effusions.', 'otherwise, the lungs are clear, without evidence of acute infiltrate or effusion.', 'no airspace disease, effusion or noncalcified nodule.', 'the lungs do not demonstrate focal infiltrate or effusion.', 'th

### Create Ground Truth labeled dataset for ML
## this code generates all types of reports and saves them to directories in outdir
### This code can be run again and again to generate as many reports because of the random selections within (add random, replace random, etc.)


### Creating the fake modification record for radiologists verification
## puts all the modified reports in one convenient table for radiologists to verify for accuracy

In [38]:
rows=set()
#run as many times this command to get a set of fake reports
rows=create_all_reportrows_withoutdicom(indir,outdir, findings, neg_map,filename_map, problem_map, 
                                                 report_to_finding_to_sentenceMap,revised_sentenceMap,rows)
rows=create_all_reportrows_withoutdicom(indir,outdir, findings, neg_map,filename_map, problem_map, 
                                                 report_to_finding_to_sentenceMap,revised_sentenceMap,rows)
rows=create_all_reportrows_withoutdicom(indir,outdir, findings, neg_map,filename_map, problem_map, 
                                                 report_to_finding_to_sentenceMap,revised_sentenceMap,rows)
#write all those unique rows into a single output file
header="Report Name\tOriginal Report\tModified Report\tSentence Added\tSentence Deleted\t AffectedFinding\t Method"
writerows(header,rows,ml_gtdir+"combined_rows.txt")

In [31]:
len(rows)

27122

In [37]:
#the combined_rows.txt was generated by repeating the above cell create_all_reportrows_withoutdicom multiple times
#to generate sufficient number of fake reports each time due to random selection of sentences
#they were concatenated (through 'cat' command to make the larger file combined_rows.txt and removing duplicates using excel (all this outside this codebase))
#df = pd.read_csv(ml_gtdir+"faked_indiana_datasetrows.txt", delimiter = "\t")
#len(df.index)
df=pd.read_csv(ml_gtdir+"combined_rows.txt", delimiter = "\t")
df.head()
#print(df.iloc[1])

,Report Name,Original Report,Modified Report,Sentence Added,Sentence Deleted,AffectedFinding,Method
0,CXR1451.txt,Left lower lobe calcified granuloma. Heart siz...,Heart size normal. No pleural effusion or pneu...,There are postsurgical changes of the left hem...,Left lower lobe calcified granuloma.,"['calcified granuloma', 'granuloma', 'volume l...",Exchange Finding
1,CXR3849.txt,Cardiomediastinal silhouettes are within norma...,Cardiomediastinal silhouettes are within norma...,Heart and mediastinum remain normal.,None,['mediastinum'],Add Random Finding
2,CXR2200.txt,Low lung volumes. Bibasilar atelectasis versus...,Bibasilar atelectasis versus scarring. Stable ...,The lungs are otherwise clear of focal infiltr...,Low lung volumes.,"['lung', 'infiltrate']",Exchange Finding
3,CXR1109.txt,Pulmonary vascularity is within normal limits....,Pulmonary vascularity is within normal limits....,Stable calcified granuloma in the left upper l...,None,['granuloma'],Add Random Finding
4,CXR1170.txt,"No pneumothorax, pleural effusion, or focal ai...",Mild cardiomegaly. Cardio mediastinal silhouet...,"Lungs are clear without focal consolidation, e...","No pneumothorax, pleural effusion, or focal ai...","['airspace disease', 'pleura', 'pneumothorax',...",Reverse Finding


# Divide the dataset into train-test-correct/deploy splits

#### using the 60-20-20 based on patients

    

In [16]:
patients_train,patients_test,patients_deploy,reportToPatientMap=prepare_splits(ml_gtdir+"combined_rows.txt",files_per_patient,ml_gtdir)

18718
Recalculated  =  1786
1071 715
1071 357 358


### Create the reports (original and Ai reports for report quality scoring from the held-out patients)

In [35]:
save_holdoutdata(ml_gtdir+"combined_rows.txt",reportToPatientMap,patients_deploy,ml_gtdir+"eportscoring2")

modif rows=  27122 27122
5317


## Create the dataset for training the classifier carrying over information on the origin 
### of the sentences so the performance could be assessed per perturbation

In [ ]:
#rowMap=create_ml_dataset(ml_gtdir+"combined_rows.txt",reportToPatientMap,patients_train,patients_test,patients_deploy,files_per_patient,ml_gtdir+"ML_datasetrows_complete.txt",True)
rowMap=create_ml_dataset(ml_gtdir+"combined_rows.txt",reportToPatientMap,patients_train,patients_test,patients_deploy,files_per_patient,ml_gtdir+"ML_datasetrows2.txt",False)


In [19]:
len(rowMap)

26463